# Scratchwork

In [1]:
from finite_algebras import make_finite_algebra, generate_algebra_mod_n
from finite_algebras import Group, Field, is_field

import numpy as np

In [2]:
import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

## Checking for Field

### Rings

In [3]:
rng = make_finite_algebra('Powerset Ring 2',
                          'Ring on powerset of {0, 1}',
                          ['{}', '{0}', '{1}', '{0, 1}'],
                          [[0, 1, 2, 3],
                           [1, 0, 3, 2],
                           [2, 3, 0, 1],
                           [3, 2, 1, 0]],
                          [[0, 0, 0, 0],
                           [0, 1, 0, 1],
                           [0, 0, 2, 2],
                           [0, 1, 2, 3]]
                         )
rng.about()


Ring: Powerset Ring 2
Instance ID: 140261554583888
Description: Ring on powerset of {0, 1}
Identity: {}
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3  {0, 1}  {0, 1}       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
Mult. Identity: {0, 1}
Mult. Commutative? Yes
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0], [0, 1, 0, 1], [0, 0, 2, 2], [0, 1, 2, 3]]


## Fields

$GF(4) = GF(2)[X]/(X^2 + X + 1)$

See Wikipedia: ["Field with four elements"](https://en.wikipedia.org/wiki/Finite_field#Field_with_four_elements)

In [4]:
elems = ['0', '1', 'a', '1+a']

add_table = [[ '0' ,  '1' ,  'a' , '1+a'],
             [ '1' ,  '0' , '1+a',  'a' ],
             [ 'a' , '1+a',  '0' ,  '1' ],
             ['1+a',  'a' ,  '1' ,  '0' ]]

mult_table = [['0',  '0' ,  '0' ,  '0' ],
              ['0',  '1' ,  'a' , '1+a'],
              ['0',  'a' , '1+a',  '1' ],
              ['0', '1+a',  '1' ,  'a' ]]

f4 = make_finite_algebra('F4',
                         'Field with 4 elements',
                         elems,
                         add_table,
                         mult_table
                        )
f4.about()


Field: F4
Instance ID: 140261559763984
Description: Field with 4 elements
Identity: 0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       0       0       1
      1       1       1       2
      2       a       a       2
      3     1+a     1+a       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
Mult. Identity: 1
Mult. Commutative? Yes
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0], [0, 1, 2, 3], [0, 2, 3, 1], [0, 3, 1, 2]]


In [5]:
f4x = Field('F4X',
            'Also a Field with 4 elements',
            elems,
            add_table,
            mult_table
           )

f4x.about()


Field: F4X
Instance ID: 140261559732304
Description: Also a Field with 4 elements
Identity: 0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       0       0       1
      1       1       1       2
      2       a       a       2
      3     1+a     1+a       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
Mult. Identity: 1
Mult. Commutative? Yes
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0], [0, 1, 2, 3], [0, 2, 3, 1], [0, 3, 1, 2]]


In [6]:
foo = f4x.mult_abelian_subgroup()
foo.about()


Group: No name
Instance ID: 140261559688720
Description: No description
Identity: 1
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       1       1       1
      1       a     1+a       3
      2     1+a       a       3
Cayley Table (showing indices):
[[0, 1, 2], [1, 2, 0], [2, 0, 1]]


In [7]:
mult_sub_grp = f4.mult_abelian_subgroup()
mult_sub_grp.about(use_table_names=True)


Group: No name
Instance ID: 140261559765392
Description: No description
Identity: 1
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       1       1       1
      1       a     1+a       3
      2     1+a       a       3
Cayley Table (showing names):
[['1', 'a', '1+a'], ['a', '1+a', '1'], ['1+a', '1', 'a']]


In [8]:
mult_sub_grp.op('a', mult_sub_grp.inv('1+a'))

'1+a'

In [9]:
div_table = [[f4.div(x, y) for y in f4.elements] for x in f4.elements]

div_table

[[None, '0', '0', '0'],
 [None, '1', '1+a', 'a'],
 [None, 'a', '1', '1+a'],
 [None, '1+a', 'a', '1']]

In [10]:
# is_field(f4)
is_field(f4.identity, f4.elements, f4.mult_table.table)

Group(
'No name',
'No description',
['1', 'a', '1+a'],
[[0, 1, 2], [1, 2, 0], [2, 0, 1]]
)

### Generate Field of Prime Order

In [11]:
f7 = generate_algebra_mod_n(7)
f7.about()


Field: F7
Instance ID: 140261559803536
Description: Autogenerated Field of integers mod 7
Identity: a0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      a0      a0       1
      1      a1      a6       7
      2      a2      a5       7
      3      a3      a4       7
      4      a4      a3       7
      5      a5      a2       7
      6      a6      a1       7
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6],
 [1, 2, 3, 4, 5, 6, 0],
 [2, 3, 4, 5, 6, 0, 1],
 [3, 4, 5, 6, 0, 1, 2],
 [4, 5, 6, 0, 1, 2, 3],
 [5, 6, 0, 1, 2, 3, 4],
 [6, 0, 1, 2, 3, 4, 5]]
Mult. Identity: a1
Mult. Commutative? Yes
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0],
 [0, 1, 2, 3, 4, 5, 6],
 [0, 2, 4, 6, 1, 3, 5],
 [0, 3, 6, 2, 5, 1, 4],
 [0, 4, 1, 5, 2, 6, 3],
 [0, 5, 3, 1, 6, 4, 2],
 [0, 6, 5, 4, 3, 2, 1]]


In [12]:
f7_add = f7.extract_additive_algebra()
f7_add.about()


Group: F7.Add
Instance ID: 140261559764880
Description: Additive-only portion of F7
Identity: a0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      a0      a0       1
      1      a1      a6       7
      2      a2      a5       7
      3      a3      a4       7
      4      a4      a3       7
      5      a5      a2       7
      6      a6      a1       7
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6],
 [1, 2, 3, 4, 5, 6, 0],
 [2, 3, 4, 5, 6, 0, 1],
 [3, 4, 5, 6, 0, 1, 2],
 [4, 5, 6, 0, 1, 2, 3],
 [5, 6, 0, 1, 2, 3, 4],
 [6, 0, 1, 2, 3, 4, 5]]


In [13]:
f7_mult = f7.extract_multiplicative_algebra()
f7_mult.about()


Monoid: F7.Mult
Instance ID: 140261559791312
Description: Multiplicative-only portion of F7
Elements: ['a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6']
Identity: a1
Associative? Yes
Commutative? Yes
Has Inverses? No
Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0],
 [0, 1, 2, 3, 4, 5, 6],
 [0, 2, 4, 6, 1, 3, 5],
 [0, 3, 6, 2, 5, 1, 4],
 [0, 4, 1, 5, 2, 6, 3],
 [0, 5, 3, 1, 6, 4, 2],
 [0, 6, 5, 4, 3, 2, 1]]


In [14]:
f7_mult_subs = f7_mult.proper_subalgebras()

print([(alg.__class__.__name__, alg.order) for alg in f7_mult_subs])

[alg.about() for alg in f7_mult_subs if isinstance(alg, Group)]

[('Group', 6), ('Group', 2), ('Monoid', 3), ('Monoid', 4), ('Monoid', 2), ('Group', 3)]

Group: F7.Mult_subalgebra_0
Instance ID: 140261559766608
Description: Subalgebra of: Multiplicative-only portion of F7
Identity: a1
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      a1      a1       1
      1      a2      a4       3
      2      a3      a5       6
      3      a4      a2       3
      4      a5      a3       6
      5      a6      a6       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5],
 [1, 3, 5, 0, 2, 4],
 [2, 5, 1, 4, 0, 3],
 [3, 0, 4, 1, 5, 2],
 [4, 2, 0, 5, 3, 1],
 [5, 4, 3, 2, 1, 0]]

Group: F7.Mult_subalgebra_1
Instance ID: 140261559765648
Description: Subalgebra of: Multiplicative-only portion of F7
Identity: a1
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      a1      a1       1
      1      a6      a6       2
Cayley Table (showing indices):
[[0, 1], [1, 0]]

Group: F7.Mult_subalgebra_5


[None, None, None]